In [ ]:
print("hello") 

hello


In [1]:
import os
from dotenv import  load_dotenv
load_dotenv()
MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
if MODEL.startswith("llama"):
   model = Ollama(model=MODEL)
   embeddings = OllamaEmbeddings()
   print(model.invoke("Tell me a joke "))



C:\Users\JACOB\AppData\Local\Temp\ipykernel_21168\2318370604.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
C:\Users\JACOB\AppData\Local\Temp\ipykernel_21168\2318370604.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings()


Of course! Here is a quick and dirty joke for you:

Why don't scientists trust atoms? Because they make up everything!

I hope you found that funny. If not, I have plenty more where that came from! Is there anything else I can help you with?


In [3]:
from langchain_core.output_parsers import StrOutputParser
parse = StrOutputParser()
chain = model | parse
print(chain.invoke("Tell me a joke "))

Of course! Here's a classic one:

Why don't scientists trust atoms?
Because they make up everything!

I hope you find it amusing!


In [4]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("alice-in-wonderland.pdf")
pages = loader.load_and_split()
pages



[Document(metadata={'source': 'alice-in-wonderland.pdf', 'page': 0, 'page_label': '1'}, page_content='Alice’s Adventures in Wonderland \n \nLewis Carroll \n \n \n \nWith illustrations by John Tenniel \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nThis .pdf file was made available through Lenny’s Alice in Wonderland site: \nwww.alice-in-wonderland.net'),
 Document(metadata={'source': 'alice-in-wonderland.pdf', 'page': 1, 'page_label': '2'}, page_content='2'),
 Document(metadata={'source': 'alice-in-wonderland.pdf', 'page': 2, 'page_label': '3'}, page_content='3\nALL in the golden afternoon \nFull leisurely we glide; \nFor both our oars, with little skill, \nBy little arms are plied, \nWhile little hands make vain pretence \nOur wanderings to guide. \n \nAh, cruel Three! In such an hour, \nBeneath such dreamy weather, \nTo beg a tale of breath too weak \nTo stir the tiniest feather! \nYet what can one poor voice avail \nAgainst three tongues together? \n \nImperio

In [5]:
from langchain.prompts import PromptTemplate
template ="""
Answer the qusetions based on the following text. if you don't know the answer, just say "I don't know"
context: {context}
question: {question}
"""
prompt = PromptTemplate.from_template(template)

print(prompt.format(context="here is the context ", question="here is the question"))


Answer the qusetions based on the following text. if you don't know the answer, just say "I don't know"
context: here is the context 
question: here is the question



In [6]:
chain = prompt | model 

In [7]:
chain.input_schema.schema()

C:\Users\JACOB\AppData\Local\Temp\ipykernel_21168\3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [8]:
chain.invoke(
    {
        "context": "my name is alice ",
        "question": "what is my name?"
    }
)

"Of course! I'd be happy to help answer your questions based on the given text. What is your question?"

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\JACOB\AIML_project\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
retriever = vectorstore.as_retriever()
retriever.invoke("rabbit")

[Document(metadata={'source': 'alice-in-wonderland.pdf', 'page': 1, 'page_label': '2'}, page_content='2'),
 Document(metadata={'source': 'alice-in-wonderland.pdf', 'page': 64, 'page_label': '65'}, page_content="65\n \n \n`Wake up, Alice dear!' said her sister; `Why, what a long sleep you've had!'  \n \n`Oh, I've had such a curious dream!' said Alice, and she told her sister, as well as she could remember \nthem, all these strange Adventures of hers that you have just been reading about; and when she had \nfinished, her sister kissed her, and said, `It WAS a curious dream, dear, certainly: but now run in to \nyour tea; it's getting late.' So Alice got up and ran off, thinking while she ran, as well she might, what a \nwonderful dream it had been.  \n \nBut her sister sat still just as she left her, leaning her head on her hand, watching the setting sun, and \nthinking of little Alice and all her wonderful Adventures, till she too began dreaming after a fashion, \nand this was her dream:

In [11]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    
)

In [13]:
questions = [
    "What is the first line of Alice's Adventures in Wonderland?",
    "What does Alice drink to shrink in size?",
    "Who gives Alice the advice, 'We're all mad here'?",
    "What happens when Alice eats the cake labeled 'Eat Me'?",
    "How does the White Rabbit react when Alice follows him?",
    "What is the relationship between the Cheshire Cat and the Queen of Hearts?",
    "How does the Mad Hatter treat Alice during the tea party?",
    "Why does Alice grow so tall inside the White Rabbit’s house?",
    "What causes the Duchess’s baby to turn into a pig?",
    "Why does Alice argue with the Caterpillar?",
    "What is Alice’s last name?",
    "Who was the author of Alice in Wonderland and The Hobbit?",
    "What happened to Alice after she woke up in Wonderland?",
    "How does Alice change as a character from the beginning to the end of the story?",
    "What does the White Rabbit symbolize in the story?",
    "Why does the Queen of Hearts constantly order people to be executed?",
    "What role does the Dormouse play in the Mad Hatter’s tea party?",
    "What lesson does Alice learn by the end of her adventure?",
    "What are the rules of the croquet game Alice plays in Wonderland?",
    "Why is the trial at the end of the book absurd?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()


Question: What is the first line of Alice's Adventures in Wonderland?
Answer: The first line of Alice's Adventures in Wonderland is: "44. The Queen's argument was, that if something wasn't done about it in less than no time she'd have everybody executed, all round."

Question: What does Alice drink to shrink in size?
Answer: I don't know. The passage doesn't mention Alice drinking anything to shrink in size.

Question: Who gives Alice the advice, 'We're all mad here'?
Answer: The line "We're all mad here" is said by the Caterpillar to Alice in Chapter 7 of Lewis Carroll's novel "Alice's Adventures in Wonderland".

Question: What happens when Alice eats the cake labeled 'Eat Me'?
Answer: I don't know. The text you provided does not mention what happens when Alice eats the cake labeled 'Eat Me'.

Question: How does the White Rabbit react when Alice follows him?
Answer: When Alice follows the White Rabbit, he cries out "Silence in the court!" and looks anxiously round to make out who is t